# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

12. Znajdź listę wszystkich filmów o tej samej długości.

In [1]:
import pickle
import numpy as np

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');
df = pd.read_sql('select title, length from film where length = 46',con=connection)
print(df)

                 title  length
0         Alien Center      46
1            Iron Moon      46
2        Kwai Homeward      46
3     Labyrinth League      46
4  Ridgemont Submarine      46


13. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [2]:
df = pd.read_sql("select first_name, last_name, city from customer INNER JOIN address on customer.address_id=address.address_id INNER JOIN city on address.city_id = city.city_id where city='Adana'",con=connection)
print(df)

#akurat w Adanie mieszka jeden klient

  first_name last_name   city
0      Larry  Thrasher  Adana


14. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [3]:
df = pd.read_sql("SELECT AVG(amount) from film LEFT OUTER JOIN inventory ON film.film_id = inventory.film_id INNER JOIN rental ON inventory.inventory_id = rental.inventory_id INNER JOIN payment ON rental.rental_id = payment.rental_id",con=connection)
print(df)

        avg
0  4.200606


15. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [4]:
df = pd.read_sql("SELECT name, COUNT(film_id) from film_category INNER JOIN category on film_category.category_id = category.category_id  GROUP BY name",con=connection)
print(df)

           name  count
0        Horror     56
1        Comedy     58
2        Sci-Fi     61
3         Drama     62
4       Foreign     73
5      Classics     57
6         Games     61
7           New     63
8        Travel     57
9         Music     51
10       Action     64
11  Documentary     68
12       Sports     74
13    Animation     66
14       Family     69
15     Children     60


16. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [5]:
df = pd.read_sql("select country, COUNT(customer_id) from customer INNER JOIN address on customer.address_id=address.address_id INNER JOIN city on address.city_id = city.city_id RIGHT OUTER JOIN country on city.country_id = country.country_id GROUP BY country",con=connection)
print(df)

                  country  count
0                Cambodia      2
1                  Turkey     15
2                 Germany      7
3              Madagascar      1
4                    Chad      1
..                    ...    ...
104                 Sudan      2
105  United Arab Emirates      3
106              Tanzania      3
107      French Polynesia      2
108              Bulgaria      2

[109 rows x 2 columns]


17. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [6]:
df = pd.read_sql("select store.store_id, count(customer_id) from store RIGHT OUTER JOIN address on store.address_id=address.address_id INNER JOIN customer on address.address_id = customer.customer_id group by store.store_id having count(customer_id) > 100 and count(customer_id) < 300",con=connection)
print(df)

# coś jest nie tak


Empty DataFrame
Columns: [store_id, count]
Index: []


In [7]:
df = pd.read_sql("select store.store_id, count(customer_id) from store RIGHT OUTER JOIN address on store.address_id=address.address_id INNER JOIN customer on address.address_id = customer.customer_id group by store.store_id",con=connection)
print(df)

   store_id  count
0       NaN    595
1       1.0      1
2       2.0      1


In [8]:
df = pd.read_sql("select store.store_id from store",con=connection)
print(df)

#większość sklepów nie ma id?

   store_id
0         1
1         2


18. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [9]:
df = pd.read_sql("select first_name, last_name, sum(length) from customer INNER JOIN rental on rental.customer_id=customer.customer_id INNER JOIN inventory on rental.inventory_id = inventory.inventory_id INNER JOIN film on inventory.film_id = film.film_id GROUP BY customer.first_name, last_name having  sum(length) > 200",con=connection)
print(df)

    first_name    last_name   sum
0        Scott      Shelley  2916
1          Tim         Cary  4476
2       Samuel       Marlow  2291
3         Vera        Mccoy  2275
4       Nelson  Christenson  2119
..         ...          ...   ...
594      Erica     Matthews  2727
595    Stanley    Scroggins  3694
596      Ellen      Simpson  2977
597      Patsy     Davidson  3513
598  Gwendolyn          May  2890

[599 rows x 3 columns]


19. Oblicz średnią wartość wypożyczenia filmu.

In [10]:
df = pd.read_sql("SELECT title, AVG(amount) from film LEFT OUTER JOIN inventory ON film.film_id = inventory.film_id INNER JOIN rental ON inventory.inventory_id = rental.inventory_id INNER JOIN payment ON rental.rental_id = payment.rental_id group by title",con=connection)
print(df)

                           title       avg
0                 Frontier Cabin  5.990000
1    Arachnophobia Rollercoaster  4.772609
2             Cruelty Unforgiven  1.156667
3         Intolerable Intentions  5.704286
4                  Monsoon Cause  6.101111
..                           ...       ...
953                  Reef Salute  2.171818
954                   Bugsy Song  5.101111
955                Orient Closer  4.754706
956          Brotherhood Blanket  4.294348
957                Boulevard Mob  3.552500

[958 rows x 2 columns]


20. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [11]:
df = pd.read_sql("SELECT name, AVG(length) from category INNER JOIN film_category on film_category.category_id = category.category_id  INNER JOIN film on film_category.film_id = film.film_id GROUP BY name",con=connection)
print(df)

           name         avg
0        Horror  112.482143
1        Comedy  115.827586
2        Sci-Fi  108.196721
3         Drama  120.838710
4       Foreign  121.698630
5      Classics  111.666667
6         Games  127.836066
7           New  111.126984
8        Travel  113.315789
9         Music  113.647059
10       Action  111.609375
11  Documentary  108.750000
12       Sports  128.202703
13    Animation  111.015152
14       Family  114.782609
15     Children  109.800000


21. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [12]:
df = pd.read_sql("SELECT name, title, MAX(length) from category INNER JOIN film_category on film_category.category_id = category.category_id  INNER JOIN film on film_category.film_id = film.film_id GROUP BY name, film.title",con=connection)
print(df)

            name             title  max
0       Children   Crooked Frogmen  143
1       Classics    Roots Remember   89
2       Classics  Jingle Sagebrush  124
3          Games   Moonwalker Fool  184
4         Sci-Fi    Bingo Talented  150
..           ...               ...  ...
995       Family    House Dynamite  109
996       Travel     Shining Roses  125
997          New   Vanished Garden  142
998        Drama      Torque Bound  179
999  Documentary     Clerks Angels  164

[1000 rows x 3 columns]


22. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [13]:
df = pd.read_sql("SELECT title, length from category INNER JOIN film_category on film_category.category_id = category.category_id  INNER JOIN film on film_category.film_id = film.film_id order BY length desc LIMIT 1",con=connection)
print(df)

           title  length
0  Chicago North     185


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [1]:
!python -m pytest

============================= test session starts =============================
platform win32 -- Python 3.8.8, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: c:\Users\48571\OneDrive - Akademia Górniczo-Hutnicza im. Stanisława Staszica w Krakowie\Dokumenty\ABD-Jakub-Prokop\lab4
plugins: anyio-2.2.0
collected 35 items

test_main.py ...........................FFF.....                         [100%]

================================== FAILURES ===================================
___________________ test_client_by_sum_length[1000-result2] ___________________

sum_min = 1000
result =     first_name last_name   sum
0        Brian     Wyman  1265
1      Antonio      Meek  1451
2        Leona    Obrien ...08
596      Tammy   Sanders  5065
597    Eleanor      Hunt  5360
598       Karl      Seal  5388

[599 rows x 3 columns]

    @pytest.mark.parametrize("sum_min,result", result_client_by_sum_length)
    def test_client_by_sum_length(sum_min:Union[int,float], result):
        if result is None:

In [1]:
print(main.film_in_category(2))

                   title                                           languge  \
0          Alter Victory  (1,"English             ","2006-02-15 10:02:19")   
1   Anaconda Confessions  (1,"English             ","2006-02-15 10:02:19")   
2         Argonauts Town  (1,"English             ","2006-02-15 10:02:19")   
3       Bikini Borrowers  (1,"English             ","2006-02-15 10:02:19")   
4       Blackout Private  (1,"English             ","2006-02-15 10:02:19")   
..                   ...                                               ...   
61           Tracy Cider  (1,"English             ","2006-02-15 10:02:19")   
62             Turn Star  (1,"English             ","2006-02-15 10:02:19")   
63            Wait Cider  (1,"English             ","2006-02-15 10:02:19")   
64           Watch Tracy  (1,"English             ","2006-02-15 10:02:19")   
65             Wonka Sea  (1,"English             ","2006-02-15 10:02:19")   

         name  
0   Animation  
1   Animation  
2   Animation  

In [15]:
print(main.number_films_in_category(2))

        name  count
0  Animation     66


In [3]:
import main
print(main.number_film_by_length(100, 200))

    length  count
0      100     12
1      101      7
2      102     11
3      103      9
4      104      6
..     ...    ...
81     181     10
82     182      6
83     183      5
84     184      8
85     185     10

[86 rows x 2 columns]


In [17]:
print(main.client_from_city('Athenai'))

      city first_name last_name
0  Athenai      Linda  Williams


In [18]:
print(main.avg_amount_by_length(48))

   length       avg
0      48  4.295389


In [3]:
import main
print(main.client_by_sum_length(1000))

    first_name last_name   sum
0        Brian     Wyman  1265
1      Antonio      Meek  1451
2        Leona    Obrien  1588
3    Katherine    Rivera  1615
4      Tiffany    Jordan  1667
..         ...       ...   ...
594      Clara      Shaw  4808
595     Wesley      Bull  4808
596      Tammy   Sanders  5065
597    Eleanor      Hunt  5360
598       Karl      Seal  5388

[599 rows x 3 columns]


In [20]:
print(main.category_statistic_length("Action"))

     name         avg   sum  min  max
0  Action  111.609375  7143   47  185
